Attempt 009 Summarization, summarization using long-t5 and then using distilbert classifier.

In [1]:
%env CUDA_VISIBLE_DEVICES=0
!pip install transformers datasets
!pip install cloud-tpu-client==0.10 torch==1.12.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl tensorboard-plugin-profile
!pip install accelerate

env: CUDA_VISIBLE_DEVICES=0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 365 kB 71.3 MB/s 
     |████████████████████████████████| 6.6 MB 70.0 MB/s 
     |████████████████████████████████| 596 kB 78.0 MB/s 
     |████████████████████████████████| 101 kB 10.2 MB/s 
     |████████████████████████████████| 212 kB 78.8 MB/s 
     |████████████████████████████████| 141 kB 77.8 MB/s 
     |████████████████████████████████| 115 kB 79.3 MB/s 
     |████████████████████████████████| 127 kB 70.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple,

In [2]:
#adapted from https://github.com/FakeNewsChallenge/fnc-1-baseline/tree/master/utils
from csv import DictReader
import random
import os
from collections import defaultdict
import sys
import os
import re
import argparse

In [3]:
#Loading dataset from summaries
class DataReaderFromPath():
    def __init__(self, name="train", path="fnc-1", is_unlabeled=False):
        self.path = path
        print("Reading data from path : " + str(path) + " " )
        summaries = name + "_summaries.csv"
        stances = name + "_stances"
        if is_unlabeled is True:
            stances = stances + "_unlabeled"
        stances = stances + ".csv"

        self.stances = self.read(stances)
        articlesSummaries = self.read(summaries)
        self.summaries = dict()

        # make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        # copy all bodies into a dictionary
        for summary in articlesSummaries:
            self.summaries[int(summary['Body ID'])] = summary['articleSummary']

        print("Total stances: " + str(len(self.stances)))
        print("Total summaries: " + str(len(self.summaries)))

    def read(self, filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows

In [4]:
d = DataReaderFromPath()

Reading data from path : fnc-1 
Total stances: 49972
Total summaries: 1683


In [5]:
# Adapted from https://github.com/FakeNewsChallenge/fnc-1/blob/master/scorer.py
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import json
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated', 'related']
RELATED = LABELS[0:3]


def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-" * line_len)
    lines.append(header)
    lines.append("-" * line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-" * line_len)
    print('\n'.join(lines))


def report_score(actual, predicted):
    score, cm = score_submission(actual, predicted)
    best_score, _ = score_submission(actual, actual)

    print_confusion_matrix(cm)
    print("Score: " + str(score) + " out of " + str(best_score) + "\t(" + str(score * 100 / best_score) + "%)")
    all_values = get_precision_recall_f1_scores(actual, predicted)

    print("--All Metrics --")
    print(json.dumps(all_values, indent=4, sort_keys=True, separators=(',', ': ')))

    return score * 100 / best_score

#Calculating accuracy precision recall f1 scores
def get_precision_recall_f1_scores(actual, predicted):
    # calculate precision scores for labels - Average None
    p, r, f1, _ = precision_recall_fscore_support(y_true=actual, y_pred=predicted, labels=LABELS, zero_division=0.0,
                                                  average=None)

    precisions = [{LABELS[index] : value} for (index, value) in enumerate(p)]
    recalls = [{LABELS[index] : value} for (index, value) in enumerate(r)]
    f1_scores = [{LABELS[index] : value} for (index, value) in enumerate(f1)]
    acc = accuracy_score(actual, predicted)
    avg_none = {"accuracy": acc, "precision": precisions, "recall": recalls, "f1": f1_scores}

    # calculate precision scores for labels - Average micro
    p, r, f1, _ = precision_recall_fscore_support(y_true=actual, y_pred=predicted, labels=LABELS, zero_division=0.0,
                                                  average='micro')

    avg_micro = {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

    # calculate precision scores for labels - Average macro
    p, r, f1, _ = precision_recall_fscore_support(y_true=actual, y_pred=predicted, labels=LABELS, zero_division=0.0,
                                                  average='macro')

    avg_macro = {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

    # calculate precision scores for labels - Average weighted
    p, r, f1, _ = precision_recall_fscore_support(y_true=actual, y_pred=predicted, labels=LABELS, zero_division=0.0,
                                                  average='weighted')

    avg_weighted = {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

    all_values = {"Each_Class": avg_none, "micro": avg_micro, "macro": avg_macro, "weighted": avg_weighted}

    return all_values

def compute_metrics(pred):
    actual = [LABELS[int(a)] for a in pred.label_ids]#pred.label_ids
    predicted = [LABELS[int(a)] for a in pred.predictions.argmax(-1)]#pred.predictions.argmax(-1)
    acc = accuracy_score(actual, predicted)
    # calculate precision scores for labels - Average macro
    p, r, f1, _ = precision_recall_fscore_support(y_true=actual, y_pred=predicted, labels=LABELS, zero_division=0.0,
                                                  average='macro')

    avg_macro = {"accuracy": acc, "precision": p, "recall": r, "f1": f1}

    return avg_macro

In [6]:
#adapted from https://github.com/FakeNewsChallenge/fnc-1-baseline/blob/master/feature_engineering.py
import os
import re
import nltk
import numpy as np
from tqdm import tqdm

def combine_headline_and_bodies(headlines, bodies):
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        joinedStr = headline + " <TitleSummarySep> " + body
        X.append(joinedStr)
    return X

In [7]:
headlines = ["The world is ending","We all are about to die"]
bodies = ["The world is ending","We all are about to die. Everyone will be dead. We will. Kill us."]
fp = combine_headline_and_bodies(headlines, bodies)
print(fp)

2it [00:00, 29959.31it/s]

['The world is ending <TitleSummarySep> The world is ending', 'We all are about to die <TitleSummarySep> We all are about to die. Everyone will be dead. We will. Kill us.']


In [8]:
#adapted from https://github.com/FakeNewsChallenge/fnc-1-baseline/blob/master/fnc_kfold.py
import sys
import numpy as np


def generate_labeled_data(stances, dataset, name):
    h, b, y = [], [], []

    for stance in stances:
        y.append((LABELS.index(stance['Stance'])))
        h.append(stance['Headline'])
        b.append(dataset.summaries[stance['Body ID']])

    X = combine_headline_and_bodies(h, b)
    print(y)

    return X, y


def generate_data_without_labels(stances, dataset, name):
    h, b= [], []

    for stance in stances:
        h.append(stance['Headline'])
        b.append(dataset.summaries[stance['Body ID']])
        
    X = combine_headline_and_bodies(h, b)
    return X

In [9]:
import sys
import numpy as np

from sklearn.ensemble import AdaBoostClassifier
import pandas as pd
from datasets import Dataset, DatasetDict

#check_version()
#parse_params()

# Load the training dataset and generate folds
d = DataReaderFromPath()
Xall, yall = generate_labeled_data(d.stances, d, "allData")

pairedDataAndLabel = list(zip(Xall, yall))
np.random.shuffle(pairedDataAndLabel)
Xall, yall = zip(*pairedDataAndLabel)
n_train = int(0.8 * len(Xall))
n_val = int(0.1 * len(Xall))


# Load the competition dataset
unlabeled_competition_dataset = DataReaderFromPath(name="competition_test", is_unlabeled=True)

Reading data from path : fnc-1 
Total stances: 49972
Total summaries: 1683


49972it [00:00, 738998.67it/s]


[3, 0, 3, 3, 1, 0, 3, 3, 0, 3, 2, 0, 3, 3, 2, 3, 2, 0, 3, 2, 3, 2, 3, 3, 0, 0, 3, 3, 3, 2, 3, 2, 3, 3, 3, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 2, 3, 3, 0, 3, 3, 3, 3, 3, 2, 0, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 0, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 2, 3, 3, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 0, 3, 3, 1, 3, 0, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 0, 3, 2, 3, 2, 3, 2, 3, 3, 0, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 2, 3, 0, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 0, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 2, 3, 2, 3, 3, 3, 1, 3, 3, 2, 1, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 1, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 2, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 2, 

In [10]:
unlabeled_competition_dataset

In [11]:
X_unlabeled = generate_data_without_labels(unlabeled_competition_dataset.stances, unlabeled_competition_dataset,"competition_unlabeled")

25413it [00:00, 866140.49it/s]


In [12]:
len(X_unlabeled)

25413

In [13]:
competition_dataset_in_batches = []
batch_size = 200
i = 1
batch = []

for data in X_unlabeled:
  batch.append(data)
  if(i == 200):
    batch_to_append = batch.copy()
    print(batch_to_append[0])
    competition_dataset_in_batches.append(batch_to_append)
    batch.clear()
    i=0

  i = i+1
  #print(i)

if len(batch) > 0:
  competition_dataset_in_batches.append(batch)
  batch = []

Ferguson riots: Pregnant woman loses eye after cops fire BEAN BAG round through car window <TitleSummarySep> Helric Fredou, 45, turned a gun on himself in his police office in Limoges. A colleague found his body at 1am on Thursday, the day after three gunmen fired at the satirical magazine's office and left 12 people dead. Union of Commissioners of the National Police confirmed Mr Fredou took his own life.
Bear attack foiled by Justin Bieber’s music: A story too good to check <TitleSummarySep> President Cristina Fernandez de Kirchhner adopted Yair Tawil as her godson. Tradition says the seventh son in a family will turn into a werewolf and eat unbaptized babies. Yair is the first Jewish boy to be adopted, as the practice only applied to Catholics until 2009.
Finally, proof that Justin Bieber IS unbearable: Russian fisherman saved from bear attack when ringtone featuring one of the pop brat's songs scares it away <TitleSummarySep> A customer says he was able to change the quality of App

In [14]:
competition_data_dict_list = []
for batch in competition_dataset_in_batches:
    data_dict =DatasetDict(
      competition=Dataset.from_dict(
          {
              "text": batch,
              "label": [3]*len(batch),
          }
      )
    )
    competition_data_dict_list.append(data_dict)

In [15]:
competition_data_dict_list[1]['competition']['text'][0]

'Bear attack foiled by Justin Bieber’s music: A story too good to check <TitleSummarySep> President Cristina Fernandez de Kirchhner adopted Yair Tawil as her godson. Tradition says the seventh son in a family will turn into a werewolf and eat unbaptized babies. Yair is the first Jewish boy to be adopted, as the practice only applied to Catholics until 2009.'

In [16]:
data_dict = DatasetDict(
    train=Dataset.from_dict(
        {
            "text": Xall[:n_train], 
            "label": yall[:n_train]
         }
    ),
    val=Dataset.from_dict(
        {
            "text": Xall[n_train : n_train + n_val],
            "label": yall[n_train : n_train + n_val],
        }
    ),
    test=Dataset.from_dict(
        {
            "text": Xall[-n_val:], 
            "label": yall[-n_val:],
         }
    )
)

In [17]:
import os, numpy as np

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

#adapted from https://github.com/Demfier/pytorch-tutorials/blob/main/bert_classifier/train.py
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)
new_token = "<TitleSummarySep>"
tokenizer.add_tokens(new_token)



1

In [18]:
def preprocess(example):
  result =  tokenizer(example["text"], max_length=200, truncation=True, padding='max_length')  
  return result

encoded_dataset = data_dict.map(preprocess, batched=True)

  0%|          | 0/40 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [19]:
encoded_competitiondataset_list = [competition_data_dict.map(preprocess, batched=True) for competition_data_dict in competition_data_dict_list]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
tokenizer.get_added_vocab()

{'<TitleSummarySep>': 30522}

In [21]:
encoded_competitiondataset_list

[DatasetDict({
    competition: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
}),
 DatasetDict({
    competition: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
}),
 DatasetDict({
    competition: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
}),
 DatasetDict({
    competition: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
}),
 DatasetDict({
    competition: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
}),
 DatasetDict({
    competition: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
}),
 DatasetDict({
    competition: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
}),

In [22]:
encoded_dataset['train']['text'][0]

"Ottawa gunman identified as 32-year-old Michael Zehaf-Bibeau <TitleSummarySep> Ahmed Abdi Godane, the leader of the Islamist militant group al-Shabaab, was killed in a US air strike earlier this week. The United States previously said that the strike inside Somalia had targeted Godane but did not know whether he had been killed. Godane's death could now lead to an internal power struggle."

In [23]:
encoded_competitiondataset_list[0]['competition']['text'][0]

"Ferguson riots: Pregnant woman loses eye after cops fire BEAN BAG round through car window <TitleSummarySep> Helric Fredou, 45, turned a gun on himself in his police office in Limoges. A colleague found his body at 1am on Thursday, the day after three gunmen fired at the satirical magazine's office and left 12 people dead. Union of Commissioners of the National Police confirmed Mr Fredou took his own life."

In [24]:
result = []

i=0
for encoded_competition_dataset in encoded_competitiondataset_list:
  predictions = encoded_competition_dataset["competition"]['label']
  print(predictions)  
  labeled_Predictions = [LABELS[int(a)] for a in predictions]

  for predictedLabel in labeled_Predictions:
      dict = {
          "Headline": unlabeled_competition_dataset.stances[i]['Headline'],
          "Body ID": unlabeled_competition_dataset.stances[i]['Body ID'],
          "Stance": predictedLabel
      }
      result.append(dict)
      i += 1

test_data = pd.DataFrame(result)
test_data.to_csv('test_answer.csv', index=False, encoding='utf-8')  # From pandas library


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [25]:
#%%

import torch_xla.distributed.xla_multiprocessing as xmp

backbone = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=4
    )
backbone.resize_token_embeddings(len(tokenizer))
WRAPPED_MODEL = xmp.MpModelWrapper(backbone)

def train_fnc(model):
  # https://huggingface.co/docs/transformers/v4.20.1/en/main_classes/trainer#transformers.TrainingArguments
  training_args = TrainingArguments(
      "checkpoints",
      evaluation_strategy="epoch",
      save_strategy="epoch",
      learning_rate=1e-5,
      warmup_ratio=0.1,
      weight_decay=0.01,
      load_best_model_at_end=True,
      num_train_epochs=50,
      metric_for_best_model="accuracy",
      per_device_eval_batch_size=128,
      per_device_train_batch_size=128,
      tpu_num_cores = 8,
  )

  trainer = Trainer(
      model,
      training_args,
      train_dataset=encoded_dataset["train"],
      eval_dataset=encoded_dataset["val"],
      tokenizer=tokenizer,
      compute_metrics=compute_metrics,
  )

  trainer.place_model_on_device = False
  trainer.train()

  trainer.save_model("nli_model/")
  tokenizer.save_pretrained("nli_model/")

  # Run on Holdout set and report the final score on the holdout set
  predictions, _, _ = trainer.predict(encoded_dataset["test"])  
  labeled_Predictions = [LABELS[int(a)] for a in np.argmax(predictions, axis=-1)]
  actual = [LABELS[int(a)] for a in encoded_dataset["test"]["label"]]
  print("Scores on the dev set")
  report_score(actual, labeled_Predictions)
  print("")
  print("")

  result = []
  i = 0
  for encoded_competition_dataset in encoded_competitiondataset_list:
    predictions, _, _ = trainer.predict(encoded_competition_dataset["competition"])  
    labeled_Predictions = [LABELS[int(a)] for a in np.argmax(predictions, axis=-1)]

    for predictedLabel in labeled_Predictions:
        dict = {
            "Headline": unlabeled_competition_dataset.stances[i]['Headline'],
            "Body ID": unlabeled_competition_dataset.stances[i]['Body ID'],
            "Stance": predictedLabel
        }
        result.append(dict)
        i += 1

  test_data = pd.DataFrame(result)
  test_data.to_csv('answer.csv', index=False, encoding='utf-8')  # From pandas library

  return trainer


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [26]:
#%%

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

def _mp_fn(index):
    device = xm.xla_device()
    # We wrap this 
    model = WRAPPED_MODEL.to(device)

    trainer = train_fnc(model)
    return trainer

trainer = xmp.spawn(_mp_fn, start_method="fork")

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 39977
  Num Epochs = 50
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 1024
  Gradient Accumulation steps = 1
  Total optimization steps = 2000


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.274273,0.657995,0.236610,0.260058,0.247477
2,No log,0.855575,0.733440,0.183360,0.250000,0.211556
3,No log,0.577867,0.767460,0.391956,0.298907,0.295495
4,No log,0.365906,0.880328,0.398402,0.476964,0.430246
5,No log,0.293927,0.888533,0.402753,0.484370,0.435428
6,No log,0.257541,0.905343,0.574099,0.542916,0.538004
7,No log,0.234805,0.915149,0.587508,0.579355,0.576868
8,No log,0.214126,0.924755,0.606552,0.607110,0.602919
9,No log,0.191338,0.934361,0.622915,0.635075,0.628504
10,No log,0.179755,0.937763,0.632178,0.642884,0.637257


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.274273,0.657995,0.236610,0.260058,0.247477
2,No log,0.855575,0.733440,0.183360,0.250000,0.211556
3,No log,0.577867,0.767460,0.391956,0.298907,0.295495
4,No log,0.365906,0.880328,0.398402,0.476964,0.430246
5,No log,0.293927,0.888533,0.402753,0.484370,0.435428
6,No log,0.257541,0.905343,0.574099,0.542916,0.538004
7,No log,0.234805,0.915149,0.587508,0.579355,0.576868
8,No log,0.214126,0.924755,0.606552,0.607110,0.602919
9,No log,0.191338,0.934361,0.622915,0.635075,0.628504
10,No log,0.179755,0.937763,0.632178,0.642884,0.637257


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


...

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 200
  Batch size = 128


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 200
  Batch size = 128


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 200
  Batch size = 128


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 200
  Batch size = 128


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 13
  Batch size = 128


Received 9466.75 points in automatic testing in fnc-1 tester.